# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 10 2022 1:30PM,239344,20,SSNA-2173662704,Sartorius Stedim North America,Released
1,Jun 10 2022 1:27PM,239343,21,399290142,"NBTY Global, Inc.",Executing
2,Jun 10 2022 1:24PM,239342,10,SO94273-BO1,"Senseonics, Incorporated",Released
3,Jun 10 2022 1:20PM,239341,21,399290088,"NBTY Global, Inc.",Released
4,Jun 10 2022 1:13PM,239340,10,CTF0011218,"Citieffe, Inc.",Released
5,Jun 10 2022 12:52PM,239339,21,399280655,"NBTY Global, Inc.",Released
6,Jun 10 2022 12:39PM,239337,21,399290097,"NBTY Global, Inc.",Released
7,Jun 10 2022 12:26PM,239336,10,MET211133,"Methapharm, Inc.",Executing
8,Jun 10 2022 12:18PM,239333,10,0085754838,ISDIN Corporation,Released
9,Jun 10 2022 12:18PM,239333,10,0085754837,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
32,239340,Released,1
33,239341,Released,1
34,239342,Released,1
35,239343,Executing,1
36,239344,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239340,NaN,NaN,1.0
239341,NaN,NaN,1.0
239342,NaN,NaN,1.0
239343,NaN,1.0,NaN
239344,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239286,0.0,0.0,1.0
239291,0.0,0.0,1.0
239292,0.0,0.0,1.0
239294,0.0,0.0,1.0
239304,0.0,6.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239286,0,0,1
239291,0,0,1
239292,0,0,1
239294,0,0,1
239304,0,6,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239286,0,0,1
1,239291,0,0,1
2,239292,0,0,1
3,239294,0,0,1
4,239304,0,6,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239286,,,1
1,239291,,,1
2,239292,,,1
3,239294,,,1
4,239304,,6,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 10 2022 1:30PM,239344,20,Sartorius Stedim North America
1,Jun 10 2022 1:27PM,239343,21,"NBTY Global, Inc."
2,Jun 10 2022 1:24PM,239342,10,"Senseonics, Incorporated"
3,Jun 10 2022 1:20PM,239341,21,"NBTY Global, Inc."
4,Jun 10 2022 1:13PM,239340,10,"Citieffe, Inc."
5,Jun 10 2022 12:52PM,239339,21,"NBTY Global, Inc."
6,Jun 10 2022 12:39PM,239337,21,"NBTY Global, Inc."
7,Jun 10 2022 12:26PM,239336,10,"Methapharm, Inc."
8,Jun 10 2022 12:18PM,239333,10,ISDIN Corporation
38,Jun 10 2022 12:18PM,239334,10,"Nextsource Biotechnology, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 10 2022 1:30PM,239344,20,Sartorius Stedim North America,,,1
1,Jun 10 2022 1:27PM,239343,21,"NBTY Global, Inc.",,1,
2,Jun 10 2022 1:24PM,239342,10,"Senseonics, Incorporated",,,1
3,Jun 10 2022 1:20PM,239341,21,"NBTY Global, Inc.",,,1
4,Jun 10 2022 1:13PM,239340,10,"Citieffe, Inc.",,,1
5,Jun 10 2022 12:52PM,239339,21,"NBTY Global, Inc.",,,1
6,Jun 10 2022 12:39PM,239337,21,"NBTY Global, Inc.",,,1
7,Jun 10 2022 12:26PM,239336,10,"Methapharm, Inc.",,1,
8,Jun 10 2022 12:18PM,239333,10,ISDIN Corporation,,,30
9,Jun 10 2022 12:18PM,239334,10,"Nextsource Biotechnology, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 1:30PM,239344,20,Sartorius Stedim North America,1,,
1,Jun 10 2022 1:27PM,239343,21,"NBTY Global, Inc.",,1,
2,Jun 10 2022 1:24PM,239342,10,"Senseonics, Incorporated",1,,
3,Jun 10 2022 1:20PM,239341,21,"NBTY Global, Inc.",1,,
4,Jun 10 2022 1:13PM,239340,10,"Citieffe, Inc.",1,,
5,Jun 10 2022 12:52PM,239339,21,"NBTY Global, Inc.",1,,
6,Jun 10 2022 12:39PM,239337,21,"NBTY Global, Inc.",1,,
7,Jun 10 2022 12:26PM,239336,10,"Methapharm, Inc.",,1,
8,Jun 10 2022 12:18PM,239333,10,ISDIN Corporation,30,,
9,Jun 10 2022 12:18PM,239334,10,"Nextsource Biotechnology, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 1:30PM,239344,20,Sartorius Stedim North America,1,,
1,Jun 10 2022 1:27PM,239343,21,"NBTY Global, Inc.",,1,
2,Jun 10 2022 1:24PM,239342,10,"Senseonics, Incorporated",1,,
3,Jun 10 2022 1:20PM,239341,21,"NBTY Global, Inc.",1,,
4,Jun 10 2022 1:13PM,239340,10,"Citieffe, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 1:30PM,239344,20,Sartorius Stedim North America,1.0,NaN,NaN
1,Jun 10 2022 1:27PM,239343,21,"NBTY Global, Inc.",NaN,1.0,NaN
2,Jun 10 2022 1:24PM,239342,10,"Senseonics, Incorporated",1.0,NaN,NaN
3,Jun 10 2022 1:20PM,239341,21,"NBTY Global, Inc.",1.0,NaN,NaN
4,Jun 10 2022 1:13PM,239340,10,"Citieffe, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 10 2022 1:30PM,239344,20,Sartorius Stedim North America,1.0,0.0,0.0
1,Jun 10 2022 1:27PM,239343,21,"NBTY Global, Inc.",0.0,1.0,0.0
2,Jun 10 2022 1:24PM,239342,10,"Senseonics, Incorporated",1.0,0.0,0.0
3,Jun 10 2022 1:20PM,239341,21,"NBTY Global, Inc.",1.0,0.0,0.0
4,Jun 10 2022 1:13PM,239340,10,"Citieffe, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1914663,40.0,22.0,12.0
12,478634,1.0,12.0,0.0
15,239304,2.0,6.0,0.0
16,478637,20.0,0.0,0.0
20,717980,15.0,14.0,1.0
21,3111147,12.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1914663,40.0,22.0,12.0
1,12,478634,1.0,12.0,0.0
2,15,239304,2.0,6.0,0.0
3,16,478637,20.0,0.0,0.0
4,20,717980,15.0,14.0,1.0
5,21,3111147,12.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,40.0,22.0,12.0
1,12,1.0,12.0,0.0
2,15,2.0,6.0,0.0
3,16,20.0,0.0,0.0
4,20,15.0,14.0,1.0
5,21,12.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,40.0
1,12,Released,1.0
2,15,Released,2.0
3,16,Released,20.0
4,20,Released,15.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,20,21
Status,,,,,,
Completed,12.0,0.0,0.0,0.0,1.0,0.0
Executing,22.0,12.0,6.0,0.0,14.0,1.0
Released,40.0,1.0,2.0,20.0,15.0,12.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,20,21
0,Completed,12.0,0.0,0.0,0.0,1.0,0.0
1,Executing,22.0,12.0,6.0,0.0,14.0,1.0
2,Released,40.0,1.0,2.0,20.0,15.0,12.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,20,21
0,Completed,12.0,0.0,0.0,0.0,1.0,0.0
1,Executing,22.0,12.0,6.0,0.0,14.0,1.0
2,Released,40.0,1.0,2.0,20.0,15.0,12.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()